In [9]:
import sys
print('Python:', sys.version)
import pandas as pd
print('Pandas:', pd.__version__)
import requests
print('Requests:', requests.__version__)

Python: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
Pandas: 0.24.2
Requests: 2.21.0


# Jak získat datum státních svátku pro dané období pro analýzu v Pandas?

V pandas existuje způsob jak pracovat s datem a celkově kalendářem. Lze např. podle připraveného kalendáře filtrovat data a provádět tzv. analýzu sezónnosti. V tomto článku se zaměřím, jak na získání seznamu státních svátku v US pro dané časové období.

Funkcionalita svátků v pandas lze udělat pomocí [Holidays / Holiday Calendars](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#holidays-holiday-calendars), která využívá třídy `AbstractHolidayCalendar` z modulu `pandas.holiday`. Modul `pandas.holiday` mimo jiné obsahuje nějaké předdefinované typy US svátků.

Pro vytvoření seznamu US svátků jednoduše rozšířím třídu `AbstractHolidayCalendar` o svůj vlastní seznam svátků v atributu třídy s názvem `rules`. Pokud chci přidat do seznamu vlastní svátek, který není defaultně v modulu `pandas.holiday`, můžu si ho vytvořit pomocí třídy `pandas.holiday.Holiday`.

Příklad vytváření seznamu US svátků:

In [3]:
import datetime as dt

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay


class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]


def get_trading_close_holidays(date_from, date_to):
    inst = USTradingCalendar()
    return inst.holidays(date_from, date_to)

In [5]:
date_from = dt.datetime(2015, 1, 1)
date_to = dt.datetime(2020, 12, 31)
holidays = get_trading_close_holidays(date_from, date_to)
holidays

DatetimeIndex(['2015-01-01', '2015-01-19', '2015-02-16', '2015-04-03',
               '2015-05-25', '2015-07-03', '2015-09-07', '2015-11-26',
               '2015-12-25', '2016-01-01', '2016-01-18', '2016-02-15',
               '2016-03-25', '2016-05-30', '2016-07-04', '2016-09-05',
               '2016-11-24', '2016-12-26', '2017-01-02', '2017-01-16',
               '2017-02-20', '2017-04-14', '2017-05-29', '2017-07-04',
               '2017-09-04', '2017-11-23', '2017-12-25', '2018-01-01',
               '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28',
               '2018-07-04', '2018-09-03', '2018-11-22', '2018-12-25',
               '2019-01-01', '2019-01-21', '2019-02-18', '2019-04-19',
               '2019-05-27', '2019-07-04', '2019-09-02', '2019-11-28',
               '2019-12-25', '2020-01-01', '2020-01-20', '2020-02-17',
               '2020-04-10', '2020-05-25', '2020-07-03', '2020-09-07',
               '2020-11-26', '2020-12-25'],
              dtype='datetime64[n

# Další alternativy

## US svátky na internetu

Krom modulu `pandas.holiday`, je možno použít stažení tabulky dat pomocí knihovnz `requests` a `pandas.read_html` například z webu CME https://www.cmegroup.com/tools-information/holiday-calendar.html, popřípadě i z Wikipedie - https://en.wikipedia.org/wiki/Public_holidays_in_the_United_States. Bohužel takto stažená data se musí ještě ručně upravit a přeparsovat.

Příklad získání US státních svátků z CME:

In [7]:
CME_HOLIDAY_CALENDAR_URL = 'https://www.cmegroup.com/tools-information/holiday-calendar.html'

import pandas as pd
import requests
r = requests.get(CME_HOLIDAY_CALENDAR_URL)
holiday_calendar = pd.read_html(r.text)[0]
holiday_calendar

,U.S. Holiday,Includes the following dates:,Excel,Compact Excel
0,"Dr. Martin Luther King, Jr.",18-22 January 2019,"Dr. Martin Luther King, Jr.","Dr. Martin Luther King, Jr."
1,President's Day,15- 19 February 2019,President's Day,President's Day
2,Good Friday,18- 22 April 2019,Good Friday,Good Friday
3,Memorial Day,24 - 28 May 2019,Memorial Day,Memorial Day
4,Independence Day,3-5 July 2019,Independence Day,Independence Day
5,Labor Day,30 August - 3 September,Labor Day,Labor Day
6,Thanksgiving,27- 29 November 2019,Thanksgiving,Thanksgiving
7,Christmas,24-26 December 2019,Christmas,Christmas
8,New Year’s,31 December 2019 - 2 January 2020,New Year’s,New Year’s


In [8]:
dates = holiday_calendar['Includes the following dates:']
dates

0                   18-22 January 2019
1                 15- 19 February 2019
2                    18- 22 April 2019
3                     24 - 28 May 2019
4                        3-5 July 2019
5              30 August - 3 September
6                 27- 29 November 2019
7                  24-26 December 2019
8    31 December 2019 - 2 January 2020
Name: Includes the following dates:, dtype: object

*Parsováním dat se v dnešním článku zabývat nebudu.*

## Package holidays

Státní svátky i pro jiné země je možné získat pomocí open source package `python-holidays`, dostupné na githubu: https://github.com/dr-prodigy/python-holidays. Tohle ale vyžaduje instalaci dalšího package a ve zdrojích se právě US svátky stahují a parsují právě z Wikipedie *(holidays v0.9.10)*.

# Zdoje

* [Dokumentace Pandas - Holidays / Holiday Calendars](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#holidays-holiday-calendars)
* [Stackoverflow - Create trading holiday calendar with Pandas](https://stackoverflow.com/questions/33094297/create-trading-holiday-calendar-with-pandas)
* [Wikipedia - Public holidays in the United States](https://en.wikipedia.org/wiki/Public_holidays_in_the_United_States)
* [CME Group Holiday Calendar](https://www.cmegroup.com/tools-information/holiday-calendar.html)
* [GitHub projekt python-holidays](https://github.com/dr-prodigy/python-holidays)
* [Managing Calendars of Holidays in Time Series Prediction Projects](https://medium.com/sbc-group-blog/every-data-scientist-likes-to-spend-more-time-on-cool-fun-things-like-exploring-new-datasets-206800b76e3b)